In [1]:
# Import packages
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')
from sklearn.ensemble import RandomForestRegressor
from IPython.display import clear_output

# Set working path
path = '/Users/martinbogaert/Desktop/NBA Data Analysis/2022-2023 Awards Project clean/'

print('MAKE SURE YOU CHANGE THE WEEK VARIABLE')

MAKE SURE YOU CHANGE THE WEEK VARIABLE


In [2]:
week = 2 # CHANGE THIS
date = datetime.now().strftime('%d %B %Y')
time = datetime.now().strftime('%H:%M:%S')
print(date + ' ; ' + time)

31 October 2022 ; 11:02:16


In [3]:
features = ['PTS','VORP','FG','WS','MP','TRB','AST','PER','BPM']#, 'GS'] ### GS
model = RandomForestRegressor(n_estimators = 100, max_features = 'auto', min_samples_leaf = 2)

# Minimum minutes per game
mp = 8
# Propotion of maximum games played
gp = 1/4

In [4]:
# Load training data
train_data = pd.read_csv(path + 'Algorithm/roy/roy_data.csv')

# Load predicted data
data = pd.read_csv(path + f'Algorithm/weekly data/week_{week}.csv') # Load up-to-date data
data = data.groupby('roy').get_group(1)

# Load weekly info
with open(path + f'Algorithm/weekly data/week_{week}.txt', 'r') as file:
            date1 = file.readline().splitlines()[0][7:]
            date2 = file.readline().splitlines()[0][7:]
            n = int(file.readline()[12:])
            
season_progress = n / 15
data.head()

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,MOV,ORtg,DRtg,NRtg,MOV/A,ORtg/A,DRtg/A,NRtg/A,roy,smoy
9,Christian Koloko,C,22,TOR,6,1,14.3,0.8,1.8,0.455,...,-1.33,110.95,112.39,-1.44,0.20,111.25,110.99,0.26,1,1
18,Kennedy Chandler,PG,20,MEM,3,0,7.7,0.7,2.0,0.333,...,-1.17,119.23,120.37,-1.14,-2.21,117.96,120.21,-2.25,1,1
22,Jake LaRavia,PF,21,MEM,5,0,18.2,2.4,4.6,0.522,...,-1.17,119.23,120.37,-1.14,-2.21,117.96,120.21,-2.25,1,1
23,Kenneth Lofton Jr.,PF,20,MEM,2,0,4.5,1.0,2.0,0.500,...,-1.17,119.23,120.37,-1.14,-2.21,117.96,120.21,-2.25,1,1
25,David Roddy,PF,21,MEM,6,0,18.7,2.2,6.3,0.342,...,-1.17,119.23,120.37,-1.14,-2.21,117.96,120.21,-2.25,1,1


In [5]:
# Set training data
X_train = train_data[features]
y_train = train_data['Share']

# Set minimum requirements (games and minutes played)
data = data[data['MP'] >= mp] # Minimum of minutes per game
data = data[data['G'] > season_progress * gp] # Minimum games played

pred = []
for step in range(0, 10) :
    print('Model ' + str(step+1) + '/10 ...')
    clear_output(wait = True)
    # Fit Machine Learning model
    model.fit(X_train, y_train)

    # Predict shares of test data
    pred.append(model.predict(data[features]))

Model 10/10 ...


In [6]:
# Assemble results DataFrame
res = data.assign(Share = [np.array(pred)[:,i].mean() for i in range(0, len(data))])
res = res.sort_values('Share', ascending = False) # Sort values by predictions
res['Rank'] = list(range(1, len(res)+1))
res['week'] = len(res) * [week] # Add week

res = res.reset_index(drop = True)[['Player', 'MP' ,'Tm', 'PER', 'VORP', 'WS', 'FG', 'PTS', 'Seed', 'Share']]
res.head(5)

,Player,MP,Tm,PER,VORP,WS,FG,PTS,Seed,Share
0,Paolo Banchero,33.1,ORL,19.6,3.514286,5.857143,7.6,22.7,15,0.716745
1,Bennedict Mathurin,27.7,IND,19.3,1.171429,5.857143,6.6,21.0,9,0.406607
2,Keegan Murray,36.5,SAC,13.1,0.000000,2.050000,7.0,18.8,13,0.278550
3,Dyson Daniels,12.5,NOP,24.7,4.100000,8.200000,2.0,5.5,5,0.233778
4,Jalen Williams,16.5,OKC,19.2,4.100000,4.100000,3.5,9.0,9,0.173502


In [7]:
res.to_csv(path + f'Results/roy/results_week_{week}.csv', index = None)

In [8]:
res.head(15)

,Player,MP,Tm,PER,VORP,WS,FG,PTS,Seed,Share
0,Paolo Banchero,33.1,ORL,19.6,3.514286,5.857143,7.6,22.7,15,0.716745
1,Bennedict Mathurin,27.7,IND,19.3,1.171429,5.857143,6.6,21.0,9,0.406607
2,Keegan Murray,36.5,SAC,13.1,0.000000,2.050000,7.0,18.8,13,0.278550
3,Dyson Daniels,12.5,NOP,24.7,4.100000,8.200000,2.0,5.5,5,0.233778
4,Jalen Williams,16.5,OKC,19.2,4.100000,4.100000,3.5,9.0,9,0.173502
5,Jabari Smith Jr.,31.1,HOU,9.6,-1.171429,1.171429,4.1,13.0,15,0.102253
6,Jaden Ivey,29.8,DET,14.1,0.000000,2.733333,5.5,15.2,12,0.085666
7,Blake Wesley,11.5,SAS,19.9,0.000000,4.100000,3.0,8.0,3,0.033968
8,Walker Kessler,16.6,UTA,18.1,0.000000,4.920000,2.0,5.8,3,0.027771
9,Shaedon Sharpe,18.5,POR,13.9,-1.366667,4.100000,3.7,9.3,1,0.021961
